Define the geometry of the problem, we assume that the domain is a rectangle. 
Everything marked with a `[*]` can be modified by user needs.

In [1]:
import numpy as np
from porepy.fracs import importer
from porepy.params.units import *

#[*] path for the fracture network file name
fracture_file_name = 'network.csv'

#[*] folder to store the simulation results
folder_name = 'results'

#[*] activate to export solution in vtu format (for ParaView)
save_solution = True

#[*] define the mesh size of the problem
mesh_size = 400 * METER

#[*] define the fracture and rock matrix permeability, unit [m^2 or Darcy]
fracture_permeability = 1e3 * DARCY
rock_matrix_permeability = 1e-2 * DARCY

#[*] define the fracture aperture, unit [m]
fracture_aperture = CENTIMETER

#[*] end time for the simulation [s]
end_time = 10 * YEAR

#[*] number of time steps
number_time_steps = 10

#[*] define the pressure boundary conditions, unit [Pa or bar] 
# use key 'left', 'right', 'top', 'bottom' to define which portion of the boundary
# NOTE: use 'left' and 'right' or 'top' and 'bottom'
dirichlet_boundary = {}
dirichlet_boundary['left'] = 1e2 * BAR
dirichlet_boundary['right'] = 0 * BAR

Run single-phase flow and transport models.

In [2]:
from darcy_and_transport_data import Simulation

# consistency check for permeability
assert fracture_permeability > 0 and rock_matrix_permeability > 0

# consistency check for fracture aperture
assert fracture_aperture > 0

# geometrical tolerance for the computations
tol = 1e-7

# pack the data for the grid generation
mesh_kwargs = {'tol': tol}
mesh_kwargs['mesh_size'] = {'mode': 'weighted', 'value': mesh_size, 'bound_value': mesh_size}

# import the fractures and create the grid
gb = importer.dfm_2d_from_csv(fracture_file_name, mesh_kwargs)
gb.compute_geometry()
gb.assign_node_ordering()

# define the computational domain (useful for boundary conditions)
# by default takes the bounding box of the fracture network
domain = gb.bounding_box(as_dict=True)

# pack the problem data in a single dictionary
data_problem = {'domain': domain, 'tol': tol, 'dirichlet_boundary': dirichlet_boundary,
                'kf': fracture_permeability, 'km': rock_matrix_permeability,
                'aperture': fracture_aperture, 
                'end_time': end_time, 'number_time_steps': number_time_steps,
                'folder_name': folder_name}

# run the simulation and save the solution
simulation = Simulation(gb, data_problem)
simulation.solve(save_as="c")

Gmsh processed file successfully


Grid creation completed. Elapsed time 0.630131721496582


Created 1 2-d grids with 12530 cells
Created 63 1-d grids with 1317 cells
Created 85 0-d grids with 85 cells




Solve elliptic model
Discretize
Done. Elapsed time 5.3368916511535645
Solve linear system using direct solver
Done. Elapsed time 0.05234217643737793
Time stepping using 10 steps
Step 0 out of 10
Step 1 out of 10
Step 2 out of 10
Step 3 out of 10
Step 4 out of 10
Step 5 out of 10
Step 6 out of 10
Step 7 out of 10
Step 8 out of 10
Step 9 out of 10


Compute production altre cose? outflow

In [6]:
from darcy_and_transport_data import compute_outflow

outflow = compute_outflow(gb)
print("Outflow ", outflow)

Outflow  1.81263113789e-07
